# 실험 05-01 BJT의 전압-전류 특성

## 공통 코드

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from scipy.signal import savgol_filter # 사비츠키-골레이(Savitzky-Golay) 필터
from numpy import nan, inf

# %matplotlib tk
# %matplotlib inline

def tangent_line(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return lambda t: d*t - d*x + f(x)

def get_simulation_result(file_name, start=0, end=-1):
    with open(file_name, encoding='cp1252') as data_file:
        lines = data_file.read()
        occurrences = lines.count('Step Information:')
        data_file.seek(0)

        line = data_file.readline()
        labels = re.split(', | ,|\t', line)
        labels = [s.strip().upper() for s in labels]

        data = {}

        if (occurrences == 0):
            for label in labels:
                data[label] = []
            
            for line in data_file:
                values = re.split(', | ,|\t', line)
                for i in range(len(values)):
                    value = float(values[i]) * 1000
                    data[labels[i]].append(value)

        else:
            labels_all =[]
            
            lines = data_file.readline() # skip first line starting with 'Step Information:'

            for idx in range(occurrences):
                labels_new = []
                for label in labels:
                    labels_new.append('(%s)@%d' % (label, idx+1))
                labels_all += labels_new

                for label_new in labels_new:
                    data[label_new] = []

                for line in data_file:
                    if (line.startswith('Step Information:')):
                        break
                    values = re.split(', | ,|\t', line)
                    for i in range(len(values)):
                        value = float(values[i]) * 1000
                        data[labels_new[i]].append(value)

            labels = labels_all

        for label in labels:
            data[label] = np.array(data[label][start:end])

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def get_oscilloscpoe_result_tektronix(file_name, start=0, end=-1):
    df = pd.read_csv(file_name, header=None, encoding='cp1252',low_memory=False)

    label_ri = df.loc[df.iloc[:,0] == 'Source', 1].index
    ci = 0
    data = {}
    data['TIME'] = df.iloc[start:end, 3].to_numpy().astype(float)
    data['TIME'] -= data['TIME'][0]
    for idx in range(df.shape[1]//6):
        label = df.iloc[label_ri, ci+1].item()
        data[label] = df.iloc[start:end,ci+4].to_numpy().astype(float)
        ci += 6

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def print_array(label, values):
    print('%s = [' % label, end='')
    for idx, vd in enumerate(values):
        print('{:11.3f}'.format(vd), end='')
        if (idx+1 != len(values)):
            print(', ', end='')
    print(']')

def print_value(label, value):
    print('%s = ' % label, end='')
    print('{:11.3f}'.format(value))

def draw_plot(xs, ys, label, style_idx, marker_num=16, scatter=False, scatter_s=2):
    linestyle  = ['-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-']
    colors     = ['blue',       'green',        'red',          'orange',       'purple',
                  'cyan',       'darkseagreen', 'brown',        'goldenrod',    'darkviolet',
                  'steelblue',  'limegreen',    'tomato',       'tan',          'deeppink',
                  'navy',       'lightgreen',   'indianred',    'khaki',        'rebeccapurple',
                  'slategray',  'forestgreen',  'orangered',    'wheat',        'orchid']
    markers    = ['o',          'v',            '<',            's',            'p',
                  'h',          '*',            'X',            'x',            '^',
                  '>',          'P',            'D',            'H',            'd',
                  '|',          '-',            '4'             '5',            '6',
                  '7',          '8',            '9 ',           '10',           '+']

    if (marker_num > 0):
        markeverys = []
        if xs[0] > xs[-1]:
            first_idx = -1
        else:
            first_idx = 0
        x_step = (xs.max() - xs.min()) / marker_num
        for idx in range(len(markers)):
            x_start = xs.min() + ((x_step / (len(markers) + 1)) * idx)
            marker_list = []
            for j in range(marker_num):
                tmp_array = np.where(xs > (x_start + (x_step * j)))
                if (len(tmp_array[0]) > 0):
                    marker_list.append(tmp_array[0][first_idx])
            markeverys.append(marker_list)

        selected_markevery = markeverys[style_idx]
        selected_marker = markers[style_idx]
    elif (marker_num == 0):
        selected_markevery = None
        selected_marker = None
    else:
        selected_markevery = None
        selected_marker = markers[style_idx]

    if (scatter):
        plt.scatter(xs, ys, color=colors[style_idx], marker=selected_marker, s=scatter_s, label=label)
    else:
        plt.plot(xs, ys, ls=linestyle[style_idx], color=colors[style_idx], marker=selected_marker, markevery=selected_markevery, label=label)
    if (label != None):
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

def set_plot_size(xrate, yrate):
    if (len(set_plot_size.g_plt_figsize) == 0):
        set_plot_size.g_plt_figsize = plt.rcParams["figure.figsize"]
    size = []
    size.append(set_plot_size.g_plt_figsize[0] * xrate)
    size.append(set_plot_size.g_plt_figsize[1] * yrate)
    plt.figure(figsize=size)

set_plot_size.g_plt_figsize = []

*******************************************************************************
## 실험 1-1 결과 분석 (실험 과정 5)

### 회로도

* 사용한 소자
    + resistor: 100 kOhm, 10 Ohm
    + NPN형 BJT: Q2N3904

![Markdown-mark](../experiment/circuit_01_01_01.jpeg)
![Markdown-mark](../experiment/circuit_01_01_02.jpeg)

### 1. 실험 결과를 가져온다.

In [ ]:
data = []

data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_01.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_02.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_03.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_04.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_05.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_01_06.csv'))

### 2. 실험 결과에서 $V_C$, $V_E$를 가져온다.

측정치를 가져올 때 1000을 곱해 단위를 밀리로 변환한다.

In [ ]:
timesl = []
vcsl = []
vesl = []
labels = [500., 1000., 2000., 3000., 4000., 5000.] # VBB_mV

for idx in range(len(labels)):
    vcsl.append(data[idx]['Sample CH1'] * 1000)
    vesl.append(data[idx]['Sample CH2'] * 1000)
    vcsl[-1] = savgol_filter(vcsl[-1], 100, 2)
    vesl[-1] = savgol_filter(vesl[-1], 100, 2)


### 3. $V_C$를 x축, $V_E$를 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('VC_mV')
plt.ylabel('VE_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
for idx in range(len(labels)):
    draw_plot(vcsl[idx], vesl[idx], 'IC (VBB=%dmV)' % labels[idx], idx, scatter=True)
plt.show()

*******************************************************************************
## 실험 1-2 결과 분석 (실험 과정 6)

### 회로도

![Markdown-mark](../experiment/circuit_01_02_01.jpeg)
![Markdown-mark](../experiment/circuit_01_02_02.jpeg)

### 1. 실험 결과를 가져온다.

In [ ]:
labels = [500., 1000., 2000., 3000., 4000., 5000.]

icsl = []

vces =      np.array([    100.000,     400.000,     800.000,    2000.000])
icsl.append(np.array([          ?,           ?,           ?,           ?])) # VBB =   500.000
icsl.append(np.array([          ?,           ?,           ?,           ?])) # VBB =  1000.000
icsl.append(np.array([          ?,           ?,           ?,           ?])) # VBB =  2000.000
icsl.append(np.array([          ?,           ?,           ?,           ?])) # VBB =  3000.000
icsl.append(np.array([          ?,           ?,           ?,           ?])) # VBB =  4000.000
icsl.append(np.array([          ?,           ?,           ?,           ?])) # VBB =  5000.000

print_array('                     VCE_mV', vces)
for idx in range(len(labels)):
    print_array('VBB_mV = {:11.3f}: IB_mA'.format(labels[idx]), icsl[idx])

### 2. $V_{CE}$을 x축, $I_C$를 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('VCE_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
for idx in range(len(labels)):
    draw_plot(vces, icsl[idx], 'IC (VBB=%d)' % labels[idx], idx, marker_num=-1)
plt.show()

*******************************************************************************
## 실험 2-1 결과 분석 (실험 과정 5)

### 회로도

![Markdown-mark](../experiment/circuit_02_01_01.jpeg)
![Markdown-mark](../experiment/circuit_02_01_02.jpeg)

### 1. 실험 결과를 가져온다.

In [ ]:
RB = 100.*1000

vbbs =      np.array([    200.000,     400.000,     600.000,     800.000,    1000.000,    2000.000,    3000.000,    4000.000])
vbes =      np.array([          ?,           ?,           ?,           ?,           ?,           ?,           ?,           ?])
ics  =      np.array([          ?,           ?,           ?,           ?,           ?,           ?,           ?,           ?])
ies  =      np.array([          ?,           ?,           ?,           ?,           ?,           ?,           ?,           ?])

ibs  = (vbbs - vbes) / RB

print_array('VBB_mV', vbbs)
print_array('VBE_mA', vbes)
print_array('IB_mA ', ibs)
print_array('IC_mA ', ics)
print_array('IE_mA ', ies)

In [ ]:
beta_dcs = np.concatenate([np.array([nan, nan, nan,]), np.array(ics[3:])/np.array(ibs[3:])])
print_array('Beta_DC ', beta_dcs)

alpha_dcs = np.concatenate([np.array([nan, nan,]), np.array(ics[2:])/np.array(ies[2:])])
print_array('Alpha_DC', alpha_dcs)

### 2. $V_{BB}$를 x축, $I_B$를 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('VBB_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vbes, ibs, 'IB (VCE=%d)' % 4000, 0, marker_num=-1)
plt.show()

### 2. $V_{BB}$를 x축, $I_C$, $I_E$를 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('VBB_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vbes, ics, 'IC (VCE=%d)' % 4000, 0, marker_num=-1)
draw_plot(vbes, ies, 'IE (VCE=%d)' % 4000, 1, marker_num=-1)
plt.show()

In [ ]:
plt.xlabel('VBB_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vbes, ics, 'IC (VCE=%d)' % 4000, 0, marker_num=-1)
draw_plot(vbes, ies, 'IE (VCE=%d)' % 4000, 1, marker_num=-1)
ax = plt.gca()
ax.set_xlim([722, 726])
ax.set_ylim([6.0, 7.0])
plt.show()

# ax.set_xlim([716, 720])
# ax.set_ylim([4.8, 5.4])


*******************************************************************************
## 실험 3-1 결과 분석 (실험 과정 5)

### 회로도

* 사용한 소자
    + resistor: 100 kOhm, 10 Ohm
    + PNP형 BJT: Q2N3905

![Markdown-mark](../experiment/circuit_03_01_01.jpeg)
![Markdown-mark](../experiment/circuit_03_01_02.jpeg)

### 1. 실험 결과를 가져온다.

In [ ]:
data = []

data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_01.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_02.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_03.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_04.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_05.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_06.csv'))

### 2. 실험 결과에서 $V_C$, $V_E$를 가져온다.

측정치를 가져올 때 1000을 곱해 단위를 밀리로 변환한다.

In [ ]:
vcsl = []
vesl = []
labels = [-500., -1000., -2000., -3000., -4000., -5000.]

for idx in range(len(labels)):
    vcsl.append(data[idx]['Sample CH1'] * 1000)
    vesl.append(data[idx]['Sample CH2'] * 1000)
    vcsl[-1] = savgol_filter(vcsl[-1], 100, 2)
    vesl[-1] = savgol_filter(vesl[-1], 100, 2)


### 3. $V_C$를 x축, $V_E$를 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('VC_mV')
plt.ylabel('VE_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
for idx in range(len(labels)):
    draw_plot(vcsl[idx], vesl[idx], 'IC (VBB=%dmV)' % labels[idx], idx, scatter=True)
plt.show()

*******************************************************************************
## 실험 3-2 결과 분석 (실험 과정 6)

### 회로도

![Markdown-mark](../experiment/circuit_03_02_01.jpeg)
![Markdown-mark](../experiment/circuit_03_02_02.jpeg)

### 1. 실험 결과를 가져온다.

In [ ]:
labels = [-500., -1000., -2000., -3000., -4000., -5000.]

icsl = []

vces =      np.array([   -100.000,    -400.000,    -800.000,   -2000.000])
icsl.append(np.array([          ?,           ?,           ?,           ?])) # VBB =   -500.000
icsl.append(np.array([          ?,           ?,           ?,           ?])) # VBB =  -1000.000
icsl.append(np.array([          ?,           ?,           ?,           ?])) # VBB =  -2000.000
icsl.append(np.array([          ?,           ?,           ?,           ?])) # VBB =  -3000.000
icsl.append(np.array([          ?,           ?,           ?,           ?])) # VBB =  -4000.000
icsl.append(np.array([          ?,           ?,           ?,           ?])) # VBB =  -5000.000

print_array('                     VCE_mV', vces)
for idx in range(len(labels)):
    print_array('VBB_mV = {:11.3f}: IB_mA'.format(labels[idx]), icsl[idx])

### 2. $V_{CE}$을 x축, $I_C$를 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('VCE_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
for idx in range(len(labels)):
    draw_plot(vces, icsl[idx], 'IC (VBB=%d)' % labels[idx], idx, marker_num=-1)
plt.show()

*******************************************************************************
## 실험 4-1 결과 분석 (실험 과정 5)

### 회로도

![Markdown-mark](../experiment/circuit_04_01_01.jpeg)
![Markdown-mark](../experiment/circuit_04_01_02.jpeg)

### 1. 실험 결과를 가져온다.

In [ ]:
RB = 100.*1000

vbbs =      np.array([   -200.000,    -400.000,    -600.000,    -800.000,   -1000.000,   -2000.000,   -3000.000,   -4000.000])
vbes =      np.array([          ?,           ?,           ?,           ?,           ?,           ?,           ?,           ?])
ics  =      np.array([          ?,           ?,           ?,           ?,           ?,           ?,           ?,           ?])
ies  =      np.array([          ?,           ?,           ?,           ?,           ?,           ?,           ?,           ?])

ibs  = (vbbs - vbes) / RB

print_array('VBB_mV', vbbs)
print_array('VBE_mA', vbes)
print_array('IB_mA ', ibs)
print_array('IC_mA ', ics)
print_array('IE_mA ', ies)

In [ ]:
beta_dcs = np.concatenate([np.array([nan, nan, nan,]), np.array(ics[3:])/np.array(ibs[3:])])
print_array('Beta_DC ', beta_dcs)

alpha_dcs = np.concatenate([np.array([nan, nan,]), np.array(ics[2:])/np.array(ies[2:])])
print_array('Alpha_DC', alpha_dcs)

### 2. $V_{BB}$를 x축, $I_B$를 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('VBB_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vbes, ibs, 'IB (VCE=%d)' % 4000, 0, marker_num=-1)
plt.show()

### 2. $V_{BB}$를 x축, $I_C$, $I_E$를 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('VBB_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vbes, ics, 'IC (VCE=%d)' % 4000, 0, marker_num=-1)
draw_plot(vbes, ies, 'IE (VCE=%d)' % 4000, 1, marker_num=-1)
plt.show()

In [ ]:
plt.xlabel('VBB_mV')
plt.ylabel('I_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(vbes, ics, 'IC (VCE=%d)' % 4000, 0, marker_num=-1)
draw_plot(vbes, ies, 'IE (VCE=%d)' % 4000, 1, marker_num=-1)
ax = plt.gca()
ax.set_xlim([-720, -716])
ax.set_ylim([-5.4, -4.8])
plt.show()
plt.show()